# Workshop Tutorial: Computing in Time or Phase

This tutorial covers the concepts of time and phase, and transforming between the two quantities.

This interactive workshop tutorial covers many of the same topics as the corresponding online tutorials:

* [Advanced: Compute Times & Phases](http://phoebe-project.org/docs/2.4/tutorials/compute_times_phases.ipynb)
* [Apsidal motion (dperdt, period vs period_anom)](http://phoebe-project.org/docs/2.4/tutorials/apsidal_motion)

# Setup

We start with the usual imports, defining the logging level, and instantiating a default binary star bundle.

In [1]:
import phoebe
from phoebe import u, c
logger = phoebe.logger(clevel='WARNING')
b = phoebe.default_binary()

# Reference time

You may have noticed while adding datasets that PHOEBE works entirely in time space. This is done to allow proper parametrization of time-dependent quantities in the system but can cause difficulties if our data are given in phase-space or if we wanted to inspect a phased light curve. For this reason, PHOEBE provides several methods to help translate between the time space and phase space.

Converting between time and phase depends on a few parameters:

* `period` (orbital period of the binary at time `t0`)
* `dpdt` (change in orbital period in time)
* `t0` (reference time-point)

The value of `t0` can follow several conventions, all of which are defined in the bundle:

* `t0_supconj`: time of superior conjunction
* `t0_perpass`: time of periastron passage
* `t0_ref`: time of the reference point w.r.t. the sky (useful for apsidal motion)

The `t0_supconj`, `t0_perpass`, and `t0_ref` parameters are defined at the orbit level rather than system level.  By default, `t0_supconj` is the free parameter, with `t0_perpass` and `t0_ref` being constrained:

In [7]:
print(b.filter(qualifier='t0*'))

ParameterSet: 6 parameters
                        t0@system: 0.0 d
C     t0_perpass@binary@component: -0.25 d
      t0_supconj@binary@component: 0.0 d
C         t0_ref@binary@component: 0.0 d
            t0_perpass@constraint: t0_supconj_to_perpass({t0_supconj@binary@component}, {period@binary@component}, {ecc@binary@component}, {per0@binary@component}, {dpdt@binary@component}, {dperdt@binary@component}, {t0@system})
                t0_ref@constraint: t0_supconj_to_ref({t0_supconj@binary@component}, {period@binary@component}, {ecc@binary@component}, {per0@binary@component}, {dpdt@binary@component}, {dperdt@binary@component}, {t0@system})


For eclipsing systems, `t0_supconj` is the handy choice because the ephemerides typically provide the time of deepest minimum as the reference point, i.e. the time of superior conjunction. For non-eclipsing systems, most frequently in astrometric solutions, orbital elements provide the periastron passage time as the reference point, so in those cases we would benefit from `t0_perpass` being independent and `t0_supconj` to be constrained. Finally, for systems with eccentric orbits and apsidal motion (`dperdt` != 0), `t0_ref` defines the reference point with respect to a fixed point in the sky rather than the orbit.

Note that, when `dperdt` != 0, the role of the orbital period also becomes ambiguous: one full revolution w.r.t. orbit (the sidereal period) is different from one full revolution w.r.t. the background stars (the anomalistic period). In particular, when `dperdt`==0:

In [8]:
print(b.filter(qualifier='period*'))

ParameterSet: 5 parameters
C        period@primary@component: 1.0 d
C      period@secondary@component: 1.0 d
          period@binary@component: 1.0 d
        period@primary@constraint: {period@binary@component} / {syncpar@primary@component}
      period@secondary@constraint: {period@binary@component} / {syncpar@secondary@component}


Here we see that there is only one period, _sidereal_; but if we introduce apsidal motion:

In [15]:
b.set_value(qualifier='dperdt', component='binary', value=(1, 'deg/day'))
print(b.filter(qualifier='period*'))

ParameterSet: 7 parameters
C        period@primary@component: 1.0 d
C      period@secondary@component: 1.0 d
          period@binary@component: 1.0 d
C    period_anom@binary@component: 1.0027855153203342 d
           period_anom@constraint: {period@binary@component} / ((((-1.000000 * {period@binary@component}) * {dperdt@binary@component}) / 6.283185307179586231995926937088) + 1.000000000000000000000000000000)
        period@primary@constraint: {period@binary@component} / {syncpar@primary@component}
      period@secondary@constraint: {period@binary@component} / {syncpar@secondary@component}


Now the distinction between the sidereal and anomalistic orbital periods is important and the anomalistic period, `period_anom`, is now an exposed parameter. By default it is constrained, and the sidereal period is used as a free parameter.

# Phase-folding

For demonstration purposes let us change the orbital period so that the times and phases are not identical:

In [16]:
b.set_value(qualifier='period', component='binary', value=2.5)

The first helper method related to times and phases is `get_ephemeris()`. We can access the current ephemeris of our system using any of the predefined `t0`s, or any custom time:

In [17]:
b.get_ephemeris(t0='t0_supconj')

{'period': 2.5, 't0': 0.0, 'dpdt': 0.0}

In [18]:
b.get_ephemeris(t0='t0_perpass')

{'period': 2.5, 't0': -0.625, 'dpdt': 0.0}

In [19]:
b.get_ephemeris(t0=5)

{'period': 2.5, 't0': 5, 'dpdt': 0.0}

The next helper method is `to_phase()`. It transforms any time (float or list/array) to phase using any of these ephemerides:

In [20]:
b.to_phase([0, 0.1], t0='t0_supconj')

array([0.  , 0.04])

In [21]:
b.to_phase([0, 0.1], t0='t0_perpass')

array([0.25, 0.29])

Finally, there is a `to_time()` method. It converts phases to times (where the returned time will be the first instance of that phase after the provided `t0`):

In [22]:
b.to_time(0.5, t0='t0_supconj')

1.25

In [23]:
b.to_time(0.5, t0=2455000)

2455001.25

Compute Phases
----------------------

As we have seen in the previous tutorial, datasets have a `compute_phases` parameter, with a constraint between `compute_times` and `compute_phases`. If we wanted to compute a model in phase-space, we can achieve this by passing `compute_phases`:

In [24]:
b.add_dataset('lc', compute_phases=phoebe.linspace(0, 1, 101), dataset='lc01')

<ParameterSet: 80 parameters | contexts: figure, constraint, dataset, compute>

In [25]:
print(b.filter(qualifier=['compute_times', 'compute_phases'], context='dataset'))

ParameterSet: 2 parameters
C      compute_times@lc01@dataset: [0.    0.025 0.05  ... 2.45  2.475 2.5  ] d
      compute_phases@lc01@dataset: [0.   0.01 0.02 ... 0.98 0.99 1.  ]


If we were to change the orbital period, that would not affect the phases:

In [26]:
b.set_value('period', component='binary', value=3.14)

In [27]:
print(b.filter(qualifier=['compute_times', 'compute_phases'], context='dataset'))

ParameterSet: 2 parameters
C      compute_times@lc01@dataset: [0.     0.0314 0.0628 ... 3.0772 3.1086 3.14  ] d
      compute_phases@lc01@dataset: [0.   0.01 0.02 ... 0.98 0.99 1.  ]


Important: if your data are phase-folded, you should **not** use this to convert times and phases (and PHOEBE will raise an error as the `times` array is required if `fluxes` or `rvs` are provided). You will need to convert your phases to times (say, by using `to_time()`):

In [17]:
phases = phoebe.linspace(0, 1, 101)
times = b.to_time(phases, t0=2459752.18750)
b.add_dataset('lc', times=times, fluxes=phoebe.linspace(1, 1, 101))

<ParameterSet: 47 parameters | contexts: dataset, figure, constraint, compute>

# Exercise

Explore the effects of `dperdt` on the anomalistic period. So far we kept our orbit circular; what happens to the times and phases if you introduce eccentricity and retain apsidal motion?

Set the orbital period of the system to something other than 1 day and `t0_supconj` to something other than 0.0.  Then add a light curve dataset such that the times sample one orbital period with 100 points.